# Tentativa #1

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier  # or any other algorithm
from pyspark.ml.evaluation import BinaryClassificationEvaluator  # or MulticlassClassificationEvaluator

## 0. Initialize Spark Session

In [3]:
spark = SparkSession.builder \
    .appName("BigDataMLProject") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 18:11:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Load Data

In [4]:
#df = spark.read.csv("CHARTEVENTS.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniCHARTEVENTS.csv", mode="overwrite", header=True)

In [5]:
chartevents = spark.read.csv("miniCHARTEVENTS.csv", header=True, inferSchema=True)
chartevents.head()

Row(ROW_ID=795, SUBJECT_ID=36, HADM_ID=165660, ICUSTAY_ID=241249, ITEMID=224663, CHARTTIME=datetime.datetime(2134, 5, 12, 12, 0), STORETIME=datetime.datetime(2134, 5, 12, 14, 44), CGID=17525, VALUE=8.0, VALUENUM=8.0, VALUEUOM=None, WARNING=0, ERROR=0, RESULTSTATUS=None, STOPPED=None)

In [6]:
print("Schema:")
chartevents.printSchema()
print("Sample data:")
chartevents.show(5)

Schema:
root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ICUSTAY_ID: integer (nullable = true)
 |-- ITEMID: integer (nullable = true)
 |-- CHARTTIME: timestamp (nullable = true)
 |-- STORETIME: timestamp (nullable = true)
 |-- CGID: integer (nullable = true)
 |-- VALUE: double (nullable = true)
 |-- VALUENUM: double (nullable = true)
 |-- VALUEUOM: string (nullable = true)
 |-- WARNING: integer (nullable = true)
 |-- ERROR: integer (nullable = true)
 |-- RESULTSTATUS: string (nullable = true)
 |-- STOPPED: string (nullable = true)

Sample data:
+------+----------+-------+----------+------+-------------------+-------------------+-----+-----+--------+--------+-------+-----+------------+-------+
|ROW_ID|SUBJECT_ID|HADM_ID|ICUSTAY_ID|ITEMID|          CHARTTIME|          STORETIME| CGID|VALUE|VALUENUM|VALUEUOM|WARNING|ERROR|RESULTSTATUS|STOPPED|
+------+----------+-------+----------+------+-------------------

In [7]:
#df = spark.read.csv("ICUSTAYS.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniICUSTAYS.csv", mode="overwrite", header=True)

In [8]:
icustays = spark.read.csv("miniICUSTAYS.csv", header=True, inferSchema=True)
icustays.head()

Row(ROW_ID=372, SUBJECT_ID=275, HADM_ID=129886, ICUSTAY_ID=219649, DBSOURCE='carevue', FIRST_CAREUNIT='CCU', LAST_CAREUNIT='CCU', FIRST_WARDID=7, LAST_WARDID=7, INTIME=datetime.datetime(2170, 10, 7, 11, 28, 53), OUTTIME=datetime.datetime(2170, 10, 14, 14, 38, 7), LOS=7.1314)

In [9]:
print("Schema:")
icustays.printSchema()
print("Sample data:")
icustays.show(5)

Schema:
root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ICUSTAY_ID: integer (nullable = true)
 |-- DBSOURCE: string (nullable = true)
 |-- FIRST_CAREUNIT: string (nullable = true)
 |-- LAST_CAREUNIT: string (nullable = true)
 |-- FIRST_WARDID: integer (nullable = true)
 |-- LAST_WARDID: integer (nullable = true)
 |-- INTIME: timestamp (nullable = true)
 |-- OUTTIME: timestamp (nullable = true)
 |-- LOS: double (nullable = true)

Sample data:
+------+----------+-------+----------+--------+--------------+-------------+------------+-----------+-------------------+-------------------+-------+
|ROW_ID|SUBJECT_ID|HADM_ID|ICUSTAY_ID|DBSOURCE|FIRST_CAREUNIT|LAST_CAREUNIT|FIRST_WARDID|LAST_WARDID|             INTIME|            OUTTIME|    LOS|
+------+----------+-------+----------+--------+--------------+-------------+------------+-----------+-------------------+-------------------+-------+
|   372|       275| 

In [10]:
#df = spark.read.csv("ADMISSIONS.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniADMISSIONS.csv", mode="overwrite", header=True)

In [11]:
admissions = spark.read.csv("miniADMISSIONS.csv", header=True, inferSchema=True)
admissions.head()

Row(ROW_ID=28, SUBJECT_ID=28, HADM_ID=162569, ADMITTIME=datetime.datetime(2177, 9, 1, 7, 15), DISCHTIME=datetime.datetime(2177, 9, 6, 16, 0), DEATHTIME=None, ADMISSION_TYPE='ELECTIVE', ADMISSION_LOCATION='PHYS REFERRAL/NORMAL DELI', DISCHARGE_LOCATION='HOME HEALTH CARE', INSURANCE='Medicare', LANGUAGE=None, RELIGION='CATHOLIC', MARITAL_STATUS='MARRIED', ETHNICITY='WHITE', EDREGTIME=None, EDOUTTIME=None, DIAGNOSIS='CORONARY ARTERY DISEASE\\CORONARY ARTERY BYPASS GRAFT/SDA', HOSPITAL_EXPIRE_FLAG=0, HAS_CHARTEVENTS_DATA=1)

In [12]:
print("Schema:")
admissions.printSchema()
print("Sample data:")
admissions.show(5)

Schema:
root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ADMITTIME: timestamp (nullable = true)
 |-- DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- DISCHARGE_LOCATION: string (nullable = true)
 |-- INSURANCE: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- RELIGION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- EDREGTIME: timestamp (nullable = true)
 |-- EDOUTTIME: timestamp (nullable = true)
 |-- DIAGNOSIS: string (nullable = true)
 |-- HOSPITAL_EXPIRE_FLAG: integer (nullable = true)
 |-- HAS_CHARTEVENTS_DATA: integer (nullable = true)

Sample data:
+------+----------+-------+-------------------+-------------------+---------+--------------+--------------------+-----------------

In [13]:
#df = spark.read.csv("LABEVENTS.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniLABEVENTS.csv", mode="overwrite", header=True)

In [14]:
labevents = spark.read.csv("miniADMISSIONS.csv", header=True, inferSchema=True)
labevents.head()
print("Schema:")
labevents.printSchema()
print("Sample data:")
labevents.show(5)

Schema:
root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ADMITTIME: timestamp (nullable = true)
 |-- DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- DISCHARGE_LOCATION: string (nullable = true)
 |-- INSURANCE: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- RELIGION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- EDREGTIME: timestamp (nullable = true)
 |-- EDOUTTIME: timestamp (nullable = true)
 |-- DIAGNOSIS: string (nullable = true)
 |-- HOSPITAL_EXPIRE_FLAG: integer (nullable = true)
 |-- HAS_CHARTEVENTS_DATA: integer (nullable = true)

Sample data:
+------+----------+-------+-------------------+-------------------+---------+--------------+--------------------+-----------------

In [15]:
#df = spark.read.csv("DIAGNOSES_ICD.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniDIAGNOSES_ICD.csv", mode="overwrite", header=True)

In [16]:
diagnoses = spark.read.csv("miniDIAGNOSES_ICD.csv", header=True, inferSchema=True)
diagnoses.head()
print("Schema:")
diagnoses.printSchema()
print("Sample data:")
diagnoses.show(5)

Schema:
root
 |-- row_id: integer (nullable = true)
 |-- subject_id: integer (nullable = true)
 |-- hadm_id: integer (nullable = true)
 |-- seq_num: integer (nullable = true)
 |-- icd9_code: string (nullable = true)

Sample data:
+------+----------+-------+-------+---------+
|row_id|subject_id|hadm_id|seq_num|icd9_code|
+------+----------+-------+-------+---------+
|112351|     10006| 142345|      8|     4240|
|112360|     10006| 142345|     17|    V5867|
|112362|     10006| 142345|     19|    41401|
|112458|     10019| 177759|      1|     0389|
|112466|     10019| 177759|      9|    78559|
+------+----------+-------+-------+---------+
only showing top 5 rows



In [17]:
#df = spark.read.csv("PROCEDURES_ICD.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniPROCEDURES_ICD.csv", mode="overwrite", header=True)

In [18]:
procedures = spark.read.csv("miniPROCEDURES_ICD.csv", header=True, inferSchema=True)
procedures.head()
print("Schema:")
procedures.printSchema()
print("Sample data:")
procedures.show(5)

Schema:
root
 |-- row_id: integer (nullable = true)
 |-- subject_id: integer (nullable = true)
 |-- hadm_id: integer (nullable = true)
 |-- seq_num: integer (nullable = true)
 |-- icd9_code: integer (nullable = true)

Sample data:
+------+----------+-------+-------+---------+
|row_id|subject_id|hadm_id|seq_num|icd9_code|
+------+----------+-------+-------+---------+
|  7354|     10130| 156668|      2|     4623|
|  9845|     10069| 146672|      5|     9604|
|  9847|     10069| 146672|      7|      331|
| 47340|     10006| 142345|      6|     9907|
| 51600|     10074| 170119|      4|     8847|
+------+----------+-------+-------+---------+
only showing top 5 rows



In [19]:
#df = spark.read.csv("INPUTEVENTS_MV.csv", header=True, inferSchema=True)

#small_df = df.sample(fraction=0.1, seed=42).limit(400)

#small_df.write.csv("miniINPUTEVENTS_MV.csv", mode="overwrite", header=True)

In [20]:
inputevents = spark.read.csv("miniINPUTEVENTS_MV.csv", header=True, inferSchema=True)
inputevents.head()
print("Schema:")
inputevents.printSchema()
print("Sample data:")
inputevents.show(5)

Schema:
root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ICUSTAY_ID: integer (nullable = true)
 |-- STARTTIME: timestamp (nullable = true)
 |-- ENDTIME: timestamp (nullable = true)
 |-- ITEMID: integer (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- AMOUNTUOM: string (nullable = true)
 |-- RATE: double (nullable = true)
 |-- RATEUOM: string (nullable = true)
 |-- STORETIME: timestamp (nullable = true)
 |-- CGID: integer (nullable = true)
 |-- ORDERID: integer (nullable = true)
 |-- LINKORDERID: integer (nullable = true)
 |-- ORDERCATEGORYNAME: string (nullable = true)
 |-- SECONDARYORDERCATEGORYNAME: string (nullable = true)
 |-- ORDERCOMPONENTTYPEDESCRIPTION: string (nullable = true)
 |-- ORDERCATEGORYDESCRIPTION: string (nullable = true)
 |-- PATIENTWEIGHT: double (nullable = true)
 |-- TOTALAMOUNT: double (nullable = true)
 |-- TOTALAMOUNTUOM: string (nullable = true)
 |-- ISOPENBAG: inte

25/05/15 18:12:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 2. Data Preprocessing

The tables were chose based on its properties regarding a persons' illness, bodily atributes or gravity of the situation. The tables were:
* ChartEVents
* Admissions
* Diagnoses
* ICUStays
* InputEvents_MV
* Procedures

### a. Choose columns to use

The majoriry of the atributes are not useful to predict the duration of a ICU stay, therefore, for a initial analysis, we will choose the ones we consider relevant.

First of all, we create temporary views of all tables into a SQL-like table.

In [53]:
chartevents.createOrReplaceTempView("chartevents")
admissions.createOrReplaceTempView("admissions")
diagnoses.createOrReplaceTempView("diagnoses")
icustays.createOrReplaceTempView("icustays")
inputevents.createOrReplaceTempView("inputevents")
labevents.createOrReplaceTempView("labevents")
procedures.createOrReplaceTempView("procedures")

Then, we create a table, using a SQL query, with the relevant features for predicting the icu stay duration, and its own column (LOS from ICUStays).

In [ ]:
inputevents.show(10)

#https://mimic.mit.edu/docs/iii/tables/inputevents_mv/

+------+----------+-------+----------+-------------------+-------------------+------+-----------+---------+----------+-------+-------------------+-----+-------+-----------+--------------------+--------------------------+-----------------------------+------------------------+-------------+-----------+--------------+---------+------------------+------------+-----------------+-----------------+-------------------+-------------------+--------------+------------+
|ROW_ID|SUBJECT_ID|HADM_ID|ICUSTAY_ID|          STARTTIME|            ENDTIME|ITEMID|     AMOUNT|AMOUNTUOM|      RATE|RATEUOM|          STORETIME| CGID|ORDERID|LINKORDERID|   ORDERCATEGORYNAME|SECONDARYORDERCATEGORYNAME|ORDERCOMPONENTTYPEDESCRIPTION|ORDERCATEGORYDESCRIPTION|PATIENTWEIGHT|TOTALAMOUNT|TOTALAMOUNTUOM|ISOPENBAG|CONTINUEINNEXTDEPT|CANCELREASON|STATUSDESCRIPTION|COMMENTS_EDITEDBY|COMMENTS_CANCELEDBY|      COMMENTS_DATE|ORIGINALAMOUNT|ORIGINALRATE|
+------+----------+-------+----------+-------------------+----------------

nao sei como hei de lidar com os tempos dos exames feitos, tipo na msm admissao conto todos os testes? faço um if para que o exame seja feito quando a pessoa ja esteja dentro a icu ou apenas quando esta no hospital?

In [ ]:
df = spark.sql("""
    SELECT 
        COALESCE(icu.subject_id, adm.subject_id, chart.subject_id) AS subject_id,
        adm.hadm_id,
        adm.admission_type,
        adm.admission_location,
        chart.itemid,
        chart.value,
        chart.error,
        diag.seq_num,
        diag.icd9_code,
        icu.first_careunit,
               

        icu.LOS
               
    FROM icustays icu
            full outer join admissions adm on adm.hadm_id=icu.hadm_id
            full outer join chartevents chart on chart.hadm_id=adm.hadm_id
            full outer join diagnoses diag on diag.hadm_id=adm.hadm_id
""")
df.show()

+----------+-------+--------------+------------------+------+-----+-----+-------+---------+--------------+-------+
|subject_id|hadm_id|admission_type|admission_location|itemid|value|error|seq_num|icd9_code|first_careunit|    LOS|
+----------+-------+--------------+------------------+------+-----+-----+-------+---------+--------------+-------+
|      1569|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL|     NULL|           CCU| 6.1209|
|       680|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL|     NULL|         TSICU| 1.5246|
|      1162|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL|     NULL|          CSRU| 1.4569|
|      2562|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL|     NULL|          CSRU| 5.8048|
|        78|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL|     NULL|          MICU| 1.4891|
|      1226|   NULL|          NULL|              NULL|  NULL| NULL| NULL|   NULL

Row(subject_id=1569, hadm_id=None, admission_type=None, admission_location=None, itemid=None, value=None, error=None, seq_num=None, icd9_code=None, first_careunit='CCU', LOS=6.1209)

merdas para justificar no relatorio:

Juntei as tabelas por admissao ao hospital e nao por pessoa, pq pode haver mais doq uma admissao por pessoa. Fiz outter join pelo msm motivo, mas dps tenho de testar isso com os dados todos e nao so com as tabelas parciais

na chartevents Each row associated with one ITEMID (e.g. 212) corresponds to an instantiation of the same measurement (e.g. heart rate) entao nao vale a pena por a coluna VALUEUOM is the unit of measurement, pq cada teste ja e feito numa unidade de medida, n vale a pena tar a dar mais do msm a ml. Vou por o erro pq se houve erro ent conta menos para a ml mas ig que e melhor q nada??? problema para a ml e nao para nos.

aqui cada chartevent, ou seja, analie/teste/raiox etc corresponde as uma linha, mais tarde temos de resolver este problema pq o objetivo e prever o tempo de internacao POR ICUSTAY_ID, ent deviamos ter apenas uma linha por admissao//icustay

na tabela dos diagnosticos SEQ_NUM provides the order in which the ICD diagnoses relate to the patient, o quao importante a doenca e no caso da pessoa, e ICD9_CODE contains the actual code corresponding to the diagnosis, ou seja a doenca id, por isso para dar join dou por paciente ou por admissao? aceito opinioes, por agr pus admissoes pq e oq pus no resto

As seen in the above table, the attributes/columns chose were: 
* Person's ID (subject_id)
* Admission ID (hadm_id)
* Admission Type (admission_type) emergency etc etc
* Admission Location (admission_location)

### b. Handle categorical columns (example)

In [23]:

categorical_cols = [col for col in df.columns if df.schema[col].dataType == StringType() and col != target_col]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index") for col in categorical_cols]

NameError: name 'StringType' is not defined

### c. Assemble features (numeric + indexed categorical)

In [ ]:
numeric_cols = [col for col in df.columns if col != target_col and col not in categorical_cols]
assembler = VectorAssembler(
    inputCols=numeric_cols + [col+"_index" for col in categorical_cols],
    outputCol="features_raw"
)

### d. Scale features (optional but recommended)

In [ ]:
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

## 3. Target variable preparation (if categorical)

In [ ]:
label_indexer = StringIndexer(inputCol=target_col, outputCol="label")

## 4. Create Pipeline

In [ ]:
pipeline_stages = indexers + [assembler, scaler, label_indexer]
preprocessing_pipeline = Pipeline(stages=pipeline_stages)

In [ ]:
processed_data = preprocessing_pipeline.fit(df).transform(df)

## 5. Train-Test Split

In [ ]:
train_data, test_data = processed_data.randomSplit([0.7, 0.3], seed=42)
print(f"Training count: {train_data.count()}")
print(f"Test count: {test_data.count()}")

## 6. Model Training

In [ ]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Train model
model = rf.fit(train_data)

## 7. Predictions

In [ ]:
predictions = model.transform(test_data)

## 8. Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="label")
auc = evaluator.evaluate(predictions)
print(f"Test AUC = {auc}")

In [ ]:
# For multiclass classification
# evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print(f"Test Accuracy = {accuracy}")

## 9. Close Session

In [ ]:
spark.stop()